In [0]:
new_york_taxis = spark.read.format("csv")\
                .option("header", "true")\
                .option("inferSchema" , "true")\
                .load("/databricks-datasets/nyctaxi/tripdata/yellow/yellow_tripdata_2019-12.csv.gz")



In [0]:
new_york_taxis.limit(5).display()

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
1,2019-12-01T00:26:58.000+0000,2019-12-01T00:41:45.000+0000,1,4.2,1,N,142,116,2,14.5,3.0,0.5,0.0,0.0,0.3,18.3,2.5
1,2019-12-01T00:12:08.000+0000,2019-12-01T00:12:14.000+0000,1,0.0,1,N,145,145,2,2.5,0.5,0.5,0.0,0.0,0.3,3.8,0.0
1,2019-12-01T00:25:53.000+0000,2019-12-01T00:26:04.000+0000,1,0.0,1,N,145,145,2,2.5,0.5,0.5,0.0,0.0,0.3,3.8,0.0
1,2019-12-01T00:12:03.000+0000,2019-12-01T00:33:19.000+0000,2,9.4,1,N,138,25,1,28.5,0.5,0.5,10.0,0.0,0.3,39.8,0.0
1,2019-12-01T00:05:27.000+0000,2019-12-01T00:16:32.000+0000,2,1.6,1,N,161,237,2,9.0,3.0,0.5,0.0,0.0,0.3,12.8,2.5


In [0]:
new_york_taxis.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



Utilize um comando spark para contar quantas linhas tem esse dataframe

In [0]:
import pyspark.sql.functions as F

# df = new_york_taxis.select("VendorID") 

# Não teve diferença no tempo de execução, selecionar 1 ou todas as colunas, para esse caso, não apresentou diferenças.

display(f' O dataframe possui {new_york_taxis.count()} linhas')

' O dataframe possui 6896317 linhas'

Agrupe os dados por número de passageiros e encontre o número de corridas de cada um dos valores.

In [0]:
df = new_york_taxis.select("passenger_count") \
                   .groupBy("passenger_count") \
                   .count() \
                   .withColumnRenamed("count", "total_taxis_trips")
  
display(df)

passenger_count,total_taxis_trips
null,51018
1,4783392
6,147925
3,303007
5,244887
9,16
4,164931
8,18
7,45
2,1075747


DESAFIO: Tente agrupar por dia (1 a 31 de Desembro) e encontre o número de
corridas para cada dia.

In [0]:
df_total_pickup_day = new_york_taxis.select(("tpep_pickup_datetime"), 
                                   F.dayofmonth(("tpep_pickup_datetime")).alias("tpep_pickup_day"))

df_total_pickup_day = df_total_pickup_day.select("tpep_pickup_day") \
                                       .groupBy("tpep_pickup_day") \
                                       .count() \
                                       .withColumnRenamed("count", "total_pickup_per_day")
# display(df_total_pickup_day)                                    

In [0]:
df_total_dropoff_day = new_york_taxis.select(("tpep_dropoff_datetime"),
                                     F.dayofmonth(("tpep_dropoff_datetime")).alias("tpep_dropoff_day"))

df_total_dropoff_day = df_total_dropoff_day.select("tpep_dropoff_day") \
                                       .groupBy("tpep_dropoff_day") \
                                       .count() \
                                       .withColumnRenamed("count", "total_dropoff_per_day")
# df_total_dropoff_day.show()

In [0]:
df_total_trips = df_total_pickup_day.join(df_total_dropoff_day, df_total_pickup_day.tpep_pickup_day == df_total_dropoff_day.tpep_dropoff_day,"inner")

df = df_total_trips.select("tpep_pickup_day", "total_pickup_per_day", "total_dropoff_per_day") \
                   .withColumnRenamed("tpep_pickup_day", "day_of_month") \
                   .withColumn("mean_trips_per_day",(((F.col("total_pickup_per_day") + F.col("total_dropoff_per_day")) / 2))) \
                   .withColumn("Diference_pickup_dropoff", F.col("total_pickup_per_day") - F.col("total_dropoff_per_day"))


display(df_total_trips)



tpep_pickup_day,total_pickup_per_day,tpep_dropoff_day,total_dropoff_per_day
31,169224,31,169719
28,172031,28,171770
26,146945,26,146475
27,169616,27,169063
12,278483,12,277745
22,179056,22,180697
1,182623,1,182507
13,278121,13,277601
6,274925,6,274131
16,236426,16,236286
